# VacationPy
---

- Module 6 Challenge: "In this deliverable, you'll use your weather data skills to plan future vacations. Also, you'll use Jupyter notebooks, the geoViews Python library, and the Geoapify API."
  
- The following notebook was developed from the provided starter code.
  
- Student name: Steph Abegg 

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Convert city names to title case (wasn't required but it looks nicer)
city_data_df["City"] = [city_name.title() for city_name in city_data_df["City"]]

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Codrington,-38.2667,141.9667,11.96,78,100,4.80,AU,1723579584
1,1,Gavrilov Posad,56.5597,40.1175,14.16,82,100,3.61,RU,1723579586
2,2,Katsuura,35.1333,140.3000,26.74,91,0,2.74,JP,1723579587
3,3,Adamstown,-25.0660,-130.1015,21.03,90,100,10.13,PN,1723579588
4,4,Nouadhibou,20.9310,-17.0347,21.99,83,75,7.20,MR,1723579589


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      frame_width = 800,
                                      frame_height = 600,
                                      size = "Humidity",
                                      tiles = "OSM",
                                      title = "Cities in Dataset, Sized by Humidity"
                                     )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# My Criteria:
# A max temperature higher than 20 degrees
# Humidity < 50%
# Cloudiness < 30%
# Wind speed less than 2.2 m/s (5mph)

ideal_city_data_df = city_data_df[(city_data_df["Max Temp"] > 20) 
                                  & (city_data_df["Humidity"] < 50) 
                                  & (city_data_df["Cloudiness"] < 30) 
                                  & (city_data_df["Wind Speed"] < 2.2) 
                                 #  & (city_data_df["Country"] == "US")
                                 ]
# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna(how="any")

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
120,120,Miles City,46.4083,-105.8406,29.54,45,0,2.06,US,1723579738
191,191,Aripuana,-9.1667,-60.6333,37.14,18,3,1.37,BR,1723579829
250,250,Nahrin,36.0649,69.1334,27.38,27,0,1.62,AF,1723579902
253,253,Zhezqazghan,47.7833,67.7667,22.26,49,0,1.61,KZ,1723579905
268,268,Sarayonu,38.2620,32.4046,24.79,30,0,0.62,TR,1723579926
296,296,Maraba,-5.3686,-49.1178,35.10,49,20,2.06,BR,1723579960
310,310,Baharly,38.4362,57.4316,31.90,46,4,0.99,TM,1723579977
337,337,Enchanted Hills,35.3368,-106.5930,33.06,28,18,0.45,US,1723580011
356,356,Halabjah,35.1778,45.9861,29.22,21,0,1.14,IQ,1723580035
367,367,Immokalee,26.4187,-81.4173,36.64,49,0,2.06,US,1723580049


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
120,Miles City,US,46.4083,-105.8406,45,
191,Aripuana,BR,-9.1667,-60.6333,18,
250,Nahrin,AF,36.0649,69.1334,27,
253,Zhezqazghan,KZ,47.7833,67.7667,49,
268,Sarayonu,TR,38.2620,32.4046,30,
296,Maraba,BR,-5.3686,-49.1178,49,
310,Baharly,TM,38.4362,57.4316,46,
337,Enchanted Hills,US,35.3368,-106.5930,28,
356,Halabjah,IQ,35.1778,45.9861,21,
367,Immokalee,US,26.4187,-81.4173,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 # hotel located within 10,000m of city coordinates
params = {"categories": "accommodation.hotel", # find hotels
          "apiKey": geoapify_key 
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" # Search places inside of the circle
    params["bias"] = f"proximity:{lng},{lat}" # Search first near the location	

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    #display(name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Miles City - nearest hotel: Historic Olive Hotel
Aripuana - nearest hotel: No hotel found
Nahrin - nearest hotel: No hotel found
Zhezqazghan - nearest hotel: No hotel found
Sarayonu - nearest hotel: İnsu otel
Maraba - nearest hotel: Portobelo
Baharly - nearest hotel: No hotel found
Enchanted Hills - nearest hotel: Santa Ana Star Casino Hotel
Halabjah - nearest hotel: موتيل سازان
Immokalee - nearest hotel: Seminole Casino Hotel Immokalee
Guajara Mirim - nearest hotel: Novo Hotel Campos
As Sulayyil - nearest hotel: No hotel found
Tambopata - nearest hotel: La Habana
Antigo - nearest hotel: Holiday Inn Express & Suites Antigo
Mhamid - nearest hotel: Hotel Kasbah Azalay


,City,Country,Lat,Lng,Humidity,Hotel Name
120,Miles City,US,46.4083,-105.8406,45,Historic Olive Hotel
191,Aripuana,BR,-9.1667,-60.6333,18,No hotel found
250,Nahrin,AF,36.0649,69.1334,27,No hotel found
253,Zhezqazghan,KZ,47.7833,67.7667,49,No hotel found
268,Sarayonu,TR,38.2620,32.4046,30,İnsu otel
296,Maraba,BR,-5.3686,-49.1178,49,Portobelo
310,Baharly,TM,38.4362,57.4316,46,No hotel found
337,Enchanted Hills,US,35.3368,-106.5930,28,Santa Ana Star Casino Hotel
356,Halabjah,IQ,35.1778,45.9861,21,موتيل سازان
367,Immokalee,US,26.4187,-81.4173,49,Seminole Casino Hotel Immokalee


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = 200, # makes less sense to soze by humidity now
                                   frame_width = 800,
                                   frame_height = 500,
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   title = "Steph's Ideal Cities to Visit (Warm, Low Humidity, Sunny, and Calm)"
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)

**Discussion:** 

The above map shows all of Steph's ideal places to visit in the set of random cities. Some of these places do not have hotels within 10 km. Steph want's to have a hotel so she can continue to work remotely and have a place to be at night. So the code below filters the dataframe to just places with hotels within 10 km, and again plots the map.

Also, because I had already filtered to cities with a low humidity and because temperature and cloudiness are also important to my criteria, it makes less sense to size by humidity. So I made all of the markers the same size.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map_2 = hotel_df[hotel_df["Hotel Name"]!="No hotel found"].hvplot.points(
                                   "Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = 200, # makes less sense to soze by humidity now
                                   frame_width = 800,
                                   frame_height = 500,
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"],
                                   title = "Steph's Ideal Cities to Visit (Warm, Low Humidity, Sunny, and Calm), \
                                   \nShowing Only Places With Hotels Within 10 km of City Coordinates"
                                  )

# Display the map
hotel_map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)